In [2]:
"""
TEMPORAL FEATURE ENGINEERING FOR BANKING CHURN/GROWTH PREDICTION
==================================================================

This guide shows how to extract and engineer time-series features from 
a Core Banking System (CBS) for ML models.

Author's Context: Victor's insight about South African banking customers
keeping money longer with trusted institutions.
"""

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [3]:
# Example SQL for extracting transaction history (12 months)
TRANSACTION_QUERY = """
SELECT 
    c.customer_id,
    a.account_id,
    t.trans_date,
    t.trans_type,
    t.amount,
    t.balance_after,
    EXTRACT(DAY FROM t.trans_date) as day_of_month,
    EXTRACT(DOW FROM t.trans_date) as day_of_week
FROM transactions t
JOIN accounts a ON t.account_id = a.account_id
JOIN customers c ON a.customer_id = c.customer_id
WHERE t.trans_date >= CURRENT_DATE - INTERVAL '12 months'
  AND a.account_type IN ('SAVINGS', 'CURRENT')  -- Focus on relevant accounts
ORDER BY c.customer_id, t.trans_date
"""

# Example: Load transaction data (in practice, this would be from your CBS)
# transactions = pd.read_sql(TRANSACTION_QUERY, db_connection)

# For this demo, let's simulate some transaction data
def generate_sample_transaction_data(n_customers=1000, n_months=12):
    """
    Simulates transaction data for demonstration.
    In production, you'd query this from CBS.
    """
    np.random.seed(42)
    
    dates = pd.date_range(end=datetime.now(), periods=n_months*30, freq='D')
    
    transactions = []
    for customer_id in range(1, n_customers + 1):
        # Simulate customer behavior patterns
        is_churner = np.random.random() < 0.2  # 20% churn rate
        
        base_balance = np.random.uniform(5000, 100000)
        n_transactions = np.random.randint(20, 200)  # Transactions per year
        
        customer_dates = np.random.choice(dates, size=n_transactions, replace=False)
        customer_dates = sorted(customer_dates)
        
        balance = base_balance
        for i, trans_date in enumerate(customer_dates):
            # Simulate different transaction patterns
            if is_churner and i > n_transactions * 0.7:
                # Churners: decreasing activity near end
                amount = np.random.uniform(-500, 100)
            else:
                # Normal: balanced deposits and withdrawals
                amount = np.random.uniform(-2000, 2000)
            
            balance += amount
            balance = max(0, balance)  # Can't go negative
            
            transactions.append({
                'customer_id': customer_id,
                'trans_date': trans_date,
                'amount': amount,
                'balance_after': balance,
                'trans_type': 'DEBIT' if amount < 0 else 'CREDIT'
            })
    
    df = pd.DataFrame(transactions)
    df['trans_date'] = pd.to_datetime(df['trans_date'])
    return df

# Generate sample data
print("Generating sample transaction data...")
transactions = generate_sample_transaction_data(n_customers=1000, n_months=12)
print(f"✓ Generated {len(transactions):,} transactions for {transactions['customer_id'].nunique()} customers")
print("\nSample transactions:")
print(transactions.head(10))


Generating sample transaction data...
✓ Generated 104,680 transactions for 1000 customers

Sample transactions:
   customer_id                 trans_date       amount  balance_after  \
0            1 2024-11-19 14:41:05.115803   511.577660   95829.436769   
1            1 2024-11-22 14:41:05.115803 -1222.904186   94606.532583   
2            1 2024-11-24 14:41:05.115803 -1716.236332   92890.296251   
3            1 2024-11-26 14:41:05.115803  -412.864691   92477.431560   
4            1 2024-11-28 14:41:05.115803 -1796.925876   90680.505684   
5            1 2024-12-04 14:41:05.115803  1546.468596   92226.974280   
6            1 2024-12-05 14:41:05.115803 -1889.532913   90337.441367   
7            1 2024-12-06 14:41:05.115803   315.459582   90652.900949   
8            1 2024-12-07 14:41:05.115803  -246.103508   90406.797441   
9            1 2024-12-08 14:41:05.115803   688.104541   91094.901982   

  trans_type  
0     CREDIT  
1      DEBIT  
2      DEBIT  
3      DEBIT  
4      DE

In [ ]:

# ============================================================================
# PART 2: TEMPORAL FEATURE ENGINEERING
# ============================================================================

print("\n" + "=" * 80)
print("ENGINEERING TEMPORAL FEATURES")
print("=" * 80)

# Calculate time-based features for each customer
temporal_features = pd.DataFrame()

# Feature 1: Balance Volatility (Victor's insight: loyal customers = stable balances)
print("\n1. Calculating balance volatility...")
balance_stats = transactions.groupby('customer_id')['balance_after'].agg([
    ('balance_std', 'std'),
    ('balance_mean', 'mean'),
    ('balance_min', 'min'),
    ('balance_max', 'max'),
    ('balance_range', lambda x: x.max() - x.min())
])

# Coefficient of variation (normalized volatility)
balance_stats['balance_cv'] = balance_stats['balance_std'] / (balance_stats['balance_mean'] + 1)
temporal_features = balance_stats

print("   ✓ Balance volatility features created")

# Feature 2: Transaction Frequency and Trends
print("\n2. Calculating transaction patterns...")
trans_freq = transactions.groupby('customer_id').agg({
    'trans_date': [
        ('total_transactions', 'count'),
        ('days_active', lambda x: (x.max() - x.min()).days),
        ('last_transaction_days_ago', lambda x: (datetime.now() - x.max()).days)
    ],
    'amount': [
        ('avg_transaction_size', 'mean'),
        ('total_transaction_volume', lambda x: x.abs().sum())
    ]
})

# Flatten column names
trans_freq.columns = ['_'.join(col).strip() for col in trans_freq.columns.values]
temporal_features = temporal_features.join(trans_freq)

# Transaction frequency (transactions per active day)
temporal_features['transaction_frequency'] = (
    temporal_features['trans_date_total_transactions'] / 
    (temporal_features['trans_date_days_active'] + 1)
)

print("   ✓ Transaction frequency features created")

# Feature 3: Deposit vs Withdrawal Patterns
print("\n3. Analyzing deposit/withdrawal behavior...")
deposit_withdrawal = transactions.groupby('customer_id').apply(
    lambda x: pd.Series({
        'total_deposits': x[x['amount'] > 0]['amount'].sum(),
        'total_withdrawals': abs(x[x['amount'] < 0]['amount'].sum()),
        'num_deposits': (x['amount'] > 0).sum(),
        'num_withdrawals': (x['amount'] < 0).sum(),
        'avg_deposit': x[x['amount'] > 0]['amount'].mean(),
        'avg_withdrawal': abs(x[x['amount'] < 0]['amount'].mean()),
    })
)

# Net flow: deposits - withdrawals
deposit_withdrawal['net_flow'] = (
    deposit_withdrawal['total_deposits'] - deposit_withdrawal['total_withdrawals']
)

# Deposit/withdrawal ratio (loyalty indicator)
deposit_withdrawal['deposit_withdrawal_ratio'] = (
    deposit_withdrawal['total_deposits'] / 
    (deposit_withdrawal['total_withdrawals'] + 1)
)

temporal_features = temporal_features.join(deposit_withdrawal)
print("   ✓ Deposit/withdrawal features created")

# Feature 4: Balance Trend (Growing or Declining?)
print("\n4. Calculating balance trends...")

def calculate_balance_trend(customer_transactions):
    """
    Calculate if customer balance is growing or declining over time.
    Returns the slope of a linear regression line.
    """
    if len(customer_transactions) < 2:
        return 0
    
    x = np.arange(len(customer_transactions))
    y = customer_transactions['balance_after'].values
    
    # Linear regression: y = mx + b, we want m (slope)
    slope = np.polyfit(x, y, 1)[0]
    return slope

balance_trend = transactions.groupby('customer_id').apply(calculate_balance_trend)
temporal_features['balance_trend'] = balance_trend

# Categorize trend
temporal_features['balance_trend_category'] = pd.cut(
    temporal_features['balance_trend'],
    bins=[-np.inf, -100, 100, np.inf],
    labels=['Declining', 'Stable', 'Growing']
)

print("   ✓ Balance trend features created")

# Feature 5: Time-Between-Transactions (Victor's insight)
print("\n5. Calculating time between large transactions...")

def calculate_large_transaction_gaps(customer_transactions, threshold=1000):
    """
    Calculate average days between large deposits and subsequent large withdrawals.
    Short gaps = transactional behavior, not loyal savings.
    """
    large_trans = customer_transactions[
        customer_transactions['amount'].abs() > threshold
    ].copy()
    
    if len(large_trans) < 2:
        return np.nan
    
    large_trans = large_trans.sort_values('trans_date')
    large_trans['days_to_next'] = large_trans['trans_date'].diff().dt.days
    
    # Focus on deposit → withdrawal pattern
    large_trans['next_is_withdrawal'] = (
        large_trans['amount'].shift(-1) < 0
    ) & (large_trans['amount'] > 0)
    
    deposit_to_withdrawal_gaps = large_trans[
        large_trans['next_is_withdrawal']
    ]['days_to_next']
    
    return deposit_to_withdrawal_gaps.mean() if len(deposit_to_withdrawal_gaps) > 0 else np.nan

large_trans_gaps = transactions.groupby('customer_id').apply(
    calculate_large_transaction_gaps
)
temporal_features['avg_deposit_to_withdrawal_days'] = large_trans_gaps

print("   ✓ Large transaction gap features created")

# Feature 6: Month-End Balance Patterns
print("\n6. Analyzing month-end balance patterns...")

transactions['day_of_month'] = transactions['trans_date'].dt.day

month_patterns = transactions.groupby('customer_id').apply(
    lambda x: pd.Series({
        'avg_month_start_balance': x[x['day_of_month'] <= 7]['balance_after'].mean(),
        'avg_month_end_balance': x[x['day_of_month'] >= 25]['balance_after'].mean(),
    })
)

# Month-end drop (indicates salary account vs savings)
month_patterns['month_end_drop'] = (
    month_patterns['avg_month_start_balance'] - 
    month_patterns['avg_month_end_balance']
)

temporal_features = temporal_features.join(month_patterns)
print("   ✓ Month-end pattern features created")

# Feature 7: Recency, Frequency, Monetary (RFM) - Banking Style
print("\n7. Creating RFM features...")

temporal_features['recency_score'] = pd.qcut(
    temporal_features['trans_date_last_transaction_days_ago'],
    q=4,
    labels=[4, 3, 2, 1],  # Lower days ago = better recency
    duplicates='drop'
).astype(float)

temporal_features['frequency_score'] = pd.qcut(
    temporal_features['transaction_frequency'],
    q=4,
    labels=[1, 2, 3, 4],
    duplicates='drop'
).astype(float)

temporal_features['monetary_score'] = pd.qcut(
    temporal_features['balance_mean'],
    q=4,
    labels=[1, 2, 3, 4],
    duplicates='drop'
).astype(float)

# Combined RFM score
temporal_features['rfm_score'] = (
    temporal_features['recency_score'] + 
    temporal_features['frequency_score'] + 
    temporal_features['monetary_score']
)

print("   ✓ RFM features created")

In [ ]:
# ============================================================================
# PART 3: FEATURE SUMMARY AND INTERPRETATION
# ============================================================================

print("\n" + "=" * 80)
print("TEMPORAL FEATURES SUMMARY")
print("=" * 80)

print(f"\nTotal features engineered: {len(temporal_features.columns)}")
print("\nFeature categories:")
print("  1. Balance volatility (5 features)")
print("  2. Transaction frequency (6 features)")
print("  3. Deposit/withdrawal patterns (8 features)")
print("  4. Balance trends (2 features)")
print("  5. Large transaction gaps (1 feature)")
print("  6. Month-end patterns (3 features)")
print("  7. RFM scores (4 features)")

print("\n" + "-" * 80)
print("Sample of engineered features:")
print("-" * 80)
print(temporal_features.head(10).to_string())

print("\n" + "-" * 80)
print("Feature Statistics:")
print("-" * 80)
print(temporal_features.describe().T)

# ============================================================================
# PART 4: REAL-WORLD USAGE
# ============================================================================

print("\n" + "=" * 80)
print("HOW TO USE IN PRODUCTION")
print("=" * 80)

print("""
STEP-BY-STEP IMPLEMENTATION FOR REACH:

1. DATA EXTRACTION (Monthly or Weekly):
   - Run SQL queries against CBS to extract last 12 months of transactions
   - Store in staging table or data warehouse
   - Schedule: Airflow, cron, or CBS batch jobs

2. FEATURE CALCULATION:
   - Run this Python script on extracted data
   - Can be done in: Python, Spark (for scale), or even SQL
   - Store features in a "customer_features" table

3. MERGE WITH STATIC FEATURES:
   # In your churn model
   df = pd.read_csv('customer_static_features.csv')  # Demographics, etc.
   temporal = pd.read_csv('customer_temporal_features.csv')  # This output
   
   df_enriched = df.merge(temporal, on='customer_id', how='left')
   
   # Now run your ML model on df_enriched

4. MODEL RETRAINING:
   - Retrain model monthly with updated features
   - A/B test new model vs old model
   - Deploy winner to production

5. REAL-TIME SCORING:
   - For real-time predictions, pre-calculate temporal features
   - Store in Redis or fast database
   - API call: get_customer_churn_risk(customer_id)
     → Fetch pre-calculated features → Score with model → Return risk

KEY INSIGHT:
The hardest part isn't the ML - it's the data engineering pipeline.
Work with your Data/IT team to set up reliable feature extraction.
Once pipeline exists, iterating on features is easy!
""")

print("\n" + "=" * 80)
print("FEATURE ENGINEERING COMPLETE")
print("=" * 80)

# Save features for later use
temporal_features.to_csv('customer_temporal_features.csv')
print("\n✓ Features saved to: customer_temporal_features.csv")
print("  Merge this with your static customer data for modeling!")